In [ ]:
# ноутбук запускался на google colab
# для ускорения работы алгоритма использовалась библиотека OpenCV с CUDA
# библиотека собрана кодом в ячейке ниже

In [ ]:
# https://towardsdatascience.com/how-to-use-opencv-with-gpu-on-colab-25594379945f

# %cd /content
# !git clone https://github.com/opencv/opencv
# !git clone https://github.com/opencv/opencv_contrib
# !mkdir /content/build
# %cd /content/build
# !cmake -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules  -DBUILD_SHARED_LIBS=OFF  -DBUILD_TESTS=OFF  -DBUILD_PERF_TESTS=OFF -DBUILD_EXAMPLES=OFF -DWITH_OPENEXR=OFF -DWITH_CUDA=ON -DWITH_CUBLAS=ON -DWITH_CUDNN=ON -DOPENCV_DNN_CUDA=ON /content/opencv
# !make -j8 install

In [ ]:
# скачиваем готовую библеотеку OpenCV с CUDA в директорию с ноутбуком
# и на всякий случай перезагружаем среду выполнения
!gdown 1T-57YCbTHKVZ9Am5S9j3WUV0Pg52Kocc

Downloading...
From: https://drive.google.com/uc?id=1T-57YCbTHKVZ9Am5S9j3WUV0Pg52Kocc
To: /content/cv2.cpython-37m-x86_64-linux-gnu.so
100% 1.17G/1.17G [00:07<00:00, 162MB/s]


In [ ]:
# restart runtime

In [ ]:
# проверяем, что все хорошо
import cv2
cv2.__version__

'4.6.0-dev'

In [ ]:
!pip freeze

absl-py==1.1.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.1
attrs==21.4.0
audioread==2.1.9
autograd==1.4
Babel==2.10.3
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.8
bokeh==2.3.3
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==2.0.7
certifi==2022.6.15
cffi==1.15.1
cftime==1.6.1
chardet==3.0.4
charset-normalizer==2.1.0
click==7.1.2
cloudpickle==1.3.0
cmake==3.22.5
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda111==9.4.0
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.30
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
deprecat==2.1.1
descartes==1.1.0
dill==0

In [ ]:
# скачиваем датасеты
# трейн разархивируем в директорию с ноутбуком
# для теста создаем отдельную директорию
!gdown 1vQVJwAjHP18sUYqCho4KQXlYElb1bHlk
!unzip -q train_dataset_train.zip
!gdown 1Wx6YSTH8CChc8TGIeF6KUNBgC4LEkd2f
!unzip -q test_dataset_test.zip -d test

Downloading...
From: https://drive.google.com/uc?id=1vQVJwAjHP18sUYqCho4KQXlYElb1bHlk
To: /content/train_dataset_train.zip
100% 1.55G/1.55G [00:18<00:00, 84.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Wx6YSTH8CChc8TGIeF6KUNBgC4LEkd2f
To: /content/test_dataset_test.zip
100% 674M/674M [00:06<00:00, 112MB/s]


In [ ]:
%matplotlib inline

import pandas as pd 
import numpy as np
import random
import math
from tqdm.notebook import tqdm
import cv2
import os
import json
import zipfile

In [ ]:
MAP_SIZE = 10496

JSON_DIR = "train/json/"
IMAGE_DIR = "train/img/"
TEST_DIR = "test/"
MAP_FILE = "original.tiff"

PROJECT_NAME = "matching"
SAVE_PATH = PROJECT_NAME+"/predict/"


if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

In [ ]:
# создаем DataFrame с информацией о центрах и углах тренировочных изображений

data_df = []
for _, _, files in os.walk(JSON_DIR):
    for x in files:
        if x.endswith(".json"):
            data = json.load(open(JSON_DIR + x))
            new_row = {'id': int(x.split(".")[0]), 
                     'x': (data["right_bottom"][0]+data["left_top"][0])//2, 
                     'y': (data["right_bottom"][1]+data["left_top"][1])//2, 
                     'angle': data["angle"]}
            data_df.append(new_row)
data_df = pd.DataFrame(data_df)
data_df = data_df.sort_values('id').set_index('id')

In [ ]:
# загрузка изображения с диска
# для ускорения используем изображения в оттенках серого
def read_image(image_file):
    img = cv2.imread(image_file, cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if img is None:
        raise ValueError(f'Failed to read {image_file}')
    return img


# функция для замены области подложки снимком
def patch_map(number, old_map):     
  
    data = json.load(open(JSON_DIR+str(number)+".json"))  
    patch = read_image(IMAGE_DIR+str(number)+".png")

    x = (data["left_top"][0] + data["right_bottom"][0]) // 2
    y = (data["left_top"][1] + data["right_bottom"][1]) // 2
    angle = data["angle"]     
    
    rect = ((x, y), (1024, 1024), angle)
    box = cv2.boxPoints(rect)
    src_pts = np.array([[0, 1023],
                        [0, 0],
                        [1023, 0],
                        [1023, 1023]], 
                       dtype="float32")
    dst_pts = box.astype("float32")
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    masked = cv2.warpPerspective(patch, M, (MAP_SIZE, MAP_SIZE))
    new_map = np.where(masked>0, masked, old_map)
    
    return new_map


def crop_center(image, h, w):
    center = (image.shape[0]/2, image.shape[1]/2)
    x = center[1] - w/2
    y = center[0] - h/2

    crop_img = image[int(y):int(y+h), int(x):int(x+w)]

    return crop_img


def rotate_image(image, angle):
    height, width = image.shape[:2]
    center = (width/2, height/2)

    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=angle, scale=1)
    rotated_image = cv2.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))

    return rotated_image


# функция вращает снимок и прогоняет его по подложке
# возвращает лучший результат
def match_piece(big_map, template, mode=cv2.TM_CCOEFF_NORMED):
    x, y = 0, 0
    best_angle = 0
    best_metric = 0
    
    gtmpl = cv2.cuda_GpuMat()
    gresult = cv2.cuda_GpuMat()

    for angle in range(0, 360, 5):
        rotated_image = rotate_image(template, -angle)
        croped_image = crop_center(rotated_image, 720, 720)

        gtmpl.upload(croped_image)
        matcher = cv2.cuda.createTemplateMatching(cv2.CV_8UC1, mode)
        gresult = matcher.match(big_map, gtmpl)
        result = gresult.download()
        
        if result.max() > best_metric:
            best_metric = result.max()
            best_angle = angle
            point = np.unravel_index(result.argmax(),result.shape)
            x, y = point[1]+720//2, point[0]+720//2

    for angle in range(best_angle-4, best_angle+5):
        rotated_image = rotate_image(template, -angle)
        croped_image = crop_center(rotated_image, 720, 720)

        gtmpl.upload(croped_image)
        matcher = cv2.cuda.createTemplateMatching(cv2.CV_8UC1, mode)
        gresult = matcher.match(big_map, gtmpl)
        result = gresult.download()
        
        if result.max() > best_metric:
            best_metric = result.max()
            best_angle = angle
            point = np.unravel_index(result.argmax(),result.shape)
            x, y = point[1]+720//2, point[0]+720//2

    if best_angle > 359:
        best_angle = best_angle - 359
    if best_angle < 0:
        best_angle = best_angle + 359

    return x, y, best_angle, best_metric


def calc_rotated_point(x_center, y_center, x, y, angle):
    
    new_x = (x - x_center) * math.cos(math.radians(angle)) - \
            (y - y_center) * math.sin(math.radians(angle)) + x_center
    
    new_y = (x - x_center) * math.sin(math.radians(angle)) + \
            (y - y_center) * math.cos(math.radians(angle)) + y_center
    
    return [int(new_x), int(new_y)]


# по центру восстанавливаем координаты углов снимка
def get_rect(x, y, angle):    
  
    data = {}
    data["left_top"] = calc_rotated_point(x, y, 
                                          x - 1024//2, y + 1024//2,      
                                          angle)
    data["right_top"] = calc_rotated_point(x, y, 
                                          x + 1024//2, y + 1024//2,      
                                          angle)
    data["right_bottom"] = calc_rotated_point(x, y, 
                                          x + 1024//2, y - 1024//2,      
                                          angle)
    data["left_bottom"] = calc_rotated_point(x, y, 
                                          x - 1024//2, y - 1024//2,      
                                          angle)
    data["angle"] = angle
    
    return data


def write_to_logfile(info):
    with open(PROJECT_NAME+"/log.txt", "a") as logfile:
        logfile.write(info)
        logfile.write("\n")
        logfile.close()


# функция прогоняет по подложкам снимок
# и сохраняет наилучший результат в json файл
def predict(maps, files):

    for filename in tqdm(files):

        template = read_image(filename)

        scores = []
        results = []
        for map in maps:

            gsrc = cv2.cuda_GpuMat()
            gsrc.upload(map)
            x, y, angle, score = match_piece(gsrc, template)
            scores.append(score)
            results.append({
                'x': x,
                'y': y,
                'angle': angle,
            })

        max_score = max(scores)
        res = results[scores.index(max_score)]

        data = get_rect(res['x'], res['y'], res['angle'])
        filename_json = filename.split("/")[-1].split(".")[0]+".json"

        with open(SAVE_PATH+filename_json, 'w') as f:
            json.dump(data, f)

        write_to_logfile(filename_json+' '+str(max_score))

        print(filename_json, 'generated', max_score)

In [ ]:
# генерируем обновленные подложки

maps = [read_image(MAP_FILE) for x in range(3)]

for num in tqdm(data_df.index):
    if num <= 400:
        maps[0] = patch_map(num, maps[0])
    elif num > 800:
        maps[2] = patch_map(num, maps[2])
    else:
        maps[1] = patch_map(num, maps[1])

  0%|          | 0/800 [00:00<?, ?it/s]

In [ ]:
# формируем список изображений для предсказания
# и список файлов для архивации

test_images = []
files_list = []
for root, _, files in os.walk(TEST_DIR):
    for x in files:
        test_images.append(root+x)
        files_list.append(SAVE_PATH+x.split(".")[0]+".json") 

In [ ]:
predict(maps, test_images)

with zipfile.ZipFile(PROJECT_NAME+'.zip', 'w') as zipMe:        
    for file in files_list:
        zipMe.write(file, file.split('/')[-1], 
                    compress_type=zipfile.ZIP_DEFLATED)